Wav2vec2를 코랩에서 사용하기 위해 필요한 트랜스포머 라이브러리를 설치하시고
 오디오 데이터를 다루기 위해 필요한 datasets souldfile, jiwer 및 관련 파이썬 라이브러리도 설치하세요.

In [1]:
!pip list

Package                  Version
------------------------ ------------
aiohappyeyeballs         2.4.6
aiohttp                  3.11.12
aiosignal                1.3.2
asttokens                3.0.0
async-timeout            5.0.1
attrs                    25.1.0
audioread                3.0.1
certifi                  2025.1.31
cffi                     1.17.1
charset-normalizer       3.4.1
click                    8.1.8
comm                     0.2.2
datasets                 3.3.1
debugpy                  1.8.12
decorator                5.1.1
dill                     0.3.8
einops                   0.8.1
exceptiongroup           1.2.2
executing                2.2.0
filelock                 3.17.0
flash_attn               2.7.4.post1
frozenlist               1.5.0
fsspec                   2024.12.0
huggingface-hub          0.29.0
idna                     3.10
ipykernel                6.29.5
ipython                  8.32.0
jedi                     0.19.2
Jinja2                   3.1.5
jiwer  

In [2]:
import torch
from transformers import Wav2Vec2Model, Wav2Vec2Processor


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Wav2Vec2Model.from_pretrained(
    # "facebook/wav2vec2-large-960h-lv60-self",
    'elgeish/wav2vec2-base-timit-asr',
    torch_dtype=torch.float16,
    attn_implementation="flash_attention_2"
).to(device)

processor = Wav2Vec2Processor

/mnt/d/workspace/colab_try/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/d/workspace/colab_try/.venv/lib/python3.10/site-packages/transformers/configuration_utils.py:315: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


In [3]:
import soundfile as sf
import torch
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

model_name = 'elgeish/wav2vec2-base-timit-asr'
# elgeish/wav2vec-base-timeit-asr
# Model 및 프로세서 불러오기
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)
model.eval()

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder)

In [4]:
# 자동 음성 인식 데이터세트 불러오기.
dataset = load_dataset('kylelovesllms/timit_asr')

In [68]:
import soundfile as sf
import IPython.display as ipd
import numpy as np
import random


# dataset['test']['file']에 있는 첫 번째 데이터를 df.read( )에 투입
# print(dataset['test']['audio'])
speech = dataset['test']['audio'][3]['array']
sampling_rate = dataset['test']['audio'][0]['sampling_rate']
ipd.Audio(speech, rate=15500)

In [69]:
inputs = processor(speech, sampling_rate=16000, return_tensors="pt", padding="longest")
# input_values = processor(dataset['test']["audio"][0]["array"], return_tensors="pt").input_values


with torch.no_grad():

    # 모델에 input.input_values 투입
    outputs = model(inputs.input_values)

    # 로짓(outputs.logits)의 차원 확인.
    print(outputs.logits.shape)

    # 로짓(output.logits)의 마지막 축 기준으로 argmax 적용
    # 가장 큰 값의 인덱스를 predicted_ids에 저장.
    predicted_ids = outputs.logits.argmax(-1)
    print(predicted_ids)
    print(f"predicted: {processor.tokenizer.batch_decode(predicted_ids)[0]}")

torch.Size([1, 153, 32])
tensor([[ 0,  0,  0,  0,  0,  0,  0,  0, 12, 12, 11, 11, 11,  5,  5,  4,  4, 11,
         11,  0,  0,  7,  7, 14,  4,  4, 22, 22,  8, 16, 16, 13, 13,  4,  4, 14,
         14,  0,  0,  0,  7,  7, 13, 13, 26, 26,  4,  4,  4, 12, 12,  0,  0,  0,
          0,  0,  0, 16, 10, 10, 10,  6,  4,  4,  0, 10, 10,  9,  9,  4,  4, 21,
         21, 13, 13,  5,  5,  5,  7,  7, 12, 12,  0,  0,  0, 22, 22,  4,  4, 18,
         18,  0,  0,  0,  0,  7,  7, 12, 12, 11, 11,  4,  4, 18, 18,  0,  0,  0,
          7,  7,  6,  6,  5,  5, 13, 13,  4,  4,  0,  0,  0,  0,  7,  7,  7,  7,
         15, 15,  0,  0, 15, 15,  4,  4, 22, 22, 22,  5,  5,  5,  5,  7,  7, 13,
         13, 13,  0,  0,  0,  0,  0,  0,  0]])
predicted: she had your dark suit in greasy wash water all year
